# Copernicus Sentinel-2 Data Search and Exploration

This Jupyter Notebook demonstrates how to search, filter, and explore Sentinel-2 satellite imagery metadata using the `phidown` library. The workflow includes:

- **Setting Search Parameters:**  
    The notebook defines parameters such as collection name (`SENTINEL-2`), product type (`S2MSI1C`), cloud cover threshold, area of interest (AOI) in WKT format, date range, and sorting options.

- **Executing the Search:**  
    An instance of `CopernicusDataSearcher` is configured with the specified parameters and used to query the Copernicus Open Access Hub for matching Sentinel-2 products.

- **Viewing Results:**  
    The resulting metadata is loaded into a pandas DataFrame (`df`). Key columns such as `Id`, `Name`, `S3Path`, and `GeoFootprint` are displayed for quick inspection.

## Variables and Data

- **aoi_wkt:** AOI polygon in WKT format. Polygon must start and end with the same point. Coordinates must be given in EPSG 4326
- **cloud_cover_threshold:** Maximum allowed cloud cover percentage.
- **collection_name:** Data collection to search (`SENTINEL-2`).
- **columns_to_display:** List of DataFrame columns to display.
- **df:** DataFrame containing search results (1000 entries, 16 columns).
- **searcher:** Configured `CopernicusDataSearcher` instance.
- **start_date, end_date:** Date range for the search.
- **product_type, orbit_direction, order_by, top:** Additional search parameters.

In [ ]:
%load_ext autoreload
%autoreload 2

# Search

You can modify the search parameters to refine your query, re-run the search, and further analyze or visualize the results as needed.

In [ ]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

aoi_wkt = "POLYGON((-95.0 29.0, -95.0 30.0, -94.0 30.0, -94.0 29.0, -95.0 29.0))"  # Define the AOI in WKT format

searcher = CopernicusDataSearcher(
    collection_name='SENTINEL-1',
    product_type='SLC',
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=aoi_wkt,  # Use the defined AOI
    start_date='2023-05-03T00:00:00.000Z',
    end_date='2024-05-03T04:00:00.000Z',
    top=1000,
    attributes={'swathIdentifier': 'S1'}, # Swath identifier Stripmap: S1, ..., S6 | 'WV1,WV2' for wavemode
)

df = searcher.execute_query()
print(f"Number of results: {len(df)}")
# Display the first few rows of the DataFrame
searcher.display_results(top_n=15)

Number of results: 0
The DataFrame is empty.


In [ ]:
import time 

time.sleep(20)

## 📥 Download Your Product

Important:
If your credentials are not deleted automatically, you must manually revoke them here:

🔗 Manage S3 Credentials: https://eodata-s3keysmanager.dataspace.copernicus.eu/panel/s3-credentials

Stay secure — always clean up your credentials!

In [ ]:
filename = 'S1A_S3_SLC__1SDH_20240430T213606_20240430T213631_053668_0684A3_FCED.SAFE'
searcher.download_product(filename)

Alternative CLI

In [ ]:
from phidown.downloader import load_credentials
# Download the product using the command line
filename = 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE'
username, password = load_credentials()
# Run the command line downloader
# Note: You need to have the phidown package installed and available in your PATH
!python -m phidown.downloader -u {username} -p {password} -eo_product_name {filename}